<a href="https://colab.research.google.com/github/watanabe3tipapa/GoogleColaboratory/blob/main/method4colab_ssh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google ColaboratoryにSSH接続する方法（ngrok）

### 【前提】　ngrokのアカウントを作成し、auth_tokenを準備してあること

### 以下の手順でセットアップしていきます。

組み込むものはお好みで（以下はサンプル）

In [ ]:
import random, string, urllib.request, json, getpass, time

>ngrokのauth_tokenをあらかじめ登録しておきます。

In [ ]:
auth_token="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [ ]:
#Generate root password
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

In [ ]:
#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

In [ ]:
#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null


Creating config file /etc/ssh/sshd_config with new version
Creating SSH2 RSA key; this may take some time ...
2048 SHA256:Pt+FR5uEmWBtStBRnp8O22hy6duF+nAHsNB38+m7X0E root@4ea8a4d2a0a8 (RSA)
Creating SSH2 ECDSA key; this may take some time ...
256 SHA256:InYyJSIyzD3oKbVUaFsoTjwHBGSJ7joB08ez4SGV6lg root@4ea8a4d2a0a8 (ECDSA)
Creating SSH2 ED25519 key; this may take some time ...
256 SHA256:yE4VGfmLCmIhejMX18mDn5ChzpJcOArnV3QcqQh2UQk root@4ea8a4d2a0a8 (ED25519)
Created symlink /etc/systemd/system/sshd.service → /lib/systemd/system/ssh.service.
Created symlink /etc/systemd/system/multi-user.target.wants/ssh.service → /lib/systemd/system/ssh.service.
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.


In [ ]:
#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

In [ ]:
#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')
time.sleep(5)

In [ ]:
#Create tunnel
get_ipython().system_raw(f"./ngrok authtoken {auth_token} && ./ngrok tcp 22 &")
time.sleep(5)

In [ ]:
!ps -aux

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.2  0.0    992     4 ?        Ss   11:22   0:00 /sbin/docker-
root           8  0.3  0.3 339816 50360 ?        Sl   11:22   0:00 /tools/node/b
root          18  0.2  0.0  35888  4872 ?        Ss   11:22   0:00 tail -n +0 -F
root          29  3.0  0.0      0     0 ?        Z    11:22   0:06 [python3] <de
root          30  0.2  0.3 160408 41136 ?        S    11:22   0:00 python3 /usr/
root          55  0.8  0.4 203824 60476 ?        Sl   11:22   0:01 /usr/bin/pyth
root          56  0.0  0.0 707652  9112 ?        Sl   11:22   0:00 /usr/local/bi
root          73  7.5  0.7 469704 93492 ?        Ssl  11:25   0:02 /usr/bin/pyth
root          93  0.4  0.1 128152 15536 ?        Sl   11:25   0:00 /usr/bin/pyth
root         110 16.6  1.0 518612 141524 ?       Sl   11:25   0:04 node /datalab
root         707  0.0  0.0  95528  8812 ?        S    11:25   0:00 /usr/sbin/ssh
root         711  0.0  0.0  39196 

In [ ]:
#Get public address and print connect command
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
  (host, port) = data['tunnels'][0]['public_url'][6:].split(':')
  print(f'SSH command: ssh -p{port} root@{host}')

#Print root password
print(f'Root password: {password}')

SSH command: ssh -p19516 root@6.tcp.ngrok.io
Root password: zpB2wixlG9cNtB9WJMZd


**上記の"出力結果"をコマンドラインに入力しSSH接続します**



---

